In [121]:
import glob
import pandas as pd
from collections import Counter
import numpy as np
import time
import datetime

In [122]:
#Read artists master data that will be joined to concatenated list of all social data
artists = pd.read_csv('./July8Data/master_artists_list.csv')

In [123]:
artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_electronic', 'facebook', 'instagram', 'twitter', 'youtube',
       'musicbrainz'],
      dtype='object')

In [124]:
artists['artist_name']

0                    Shakira
1                    Beyoncé
2             Jennifer Lopez
3                Miley Cyrus
4          Justin Timberlake
5                      Adele
6                Chris Brown
7             Britney Spears
8           Enrique Iglesias
9                    Pitbull
10                    Maluma
11                 Nicky Jam
12                      Pink
13             Avril Lavigne
14             Calvin Harris
15              Romeo Santos
16                      Zayn
17               Alicia Keys
18              Harry Styles
19                     Usher
20              Ricky Martin
21              Mariah Carey
22              Prince Royce
23        Christina Aguilera
24                       Sia
25             Martin Garrix
26                Little Mix
27           Camilla Cabello
28                    Thalia
29             Fifth Harmony
               ...          
111                       MØ
112           Gloria Estefan
113    New Kids on the Block
114           

In [125]:
#Read all social data files in
files = ['./July8Data/twitter_features.csv', './July8Data/youtube_to_combine.csv', './July8Data/facebook_7yrs.csv']

In [126]:
df_list = [pd.read_csv(file) for file in files]
df_list.append(pd.read_pickle('./July8Data/instagram_7yrs_cleaned.pkl'))

C:\Users\oldye\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [127]:
sum = 0
for d in df_list:
    print("num columns", len(d.columns))
    sum+=d.shape[0]
    print(d.shape)
    print(d.columns)

print("number of rows in 4 dataframes: ", sum)

num columns 22
(324396, 22)
Index(['Unnamed: 0', 'Twitter_Handle', 'count_of_followers', 'created',
       'description', 'crowdtangle_score', 'description_length',
       'hashtag_count', 'type_link', 'type_photo', 'type_text', 'type_video',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'time_since_last_post', 'within_week_release',
       'within_month_release', 'social_engagement_score', 'last_7_days',
       'last_30_days'],
      dtype='object')
num columns 23
(24237, 23)
Index(['Unnamed: 0', 'artist_name', 'count_of_followers', 'created',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'crowdtangle_score', 'description',
       'description_length', 'hashtag_count', 'last_30_days', 'last_7_days',
       'social_engagement_score', 'time_of_day', 'time_since_last_post',
       'type_link', 'type_photo', 'type_text', 'type_video',
       'within_month_release', 'within_week_release'],
     

In [128]:
df_list[1].columns

Index(['Unnamed: 0', 'artist_name', 'count_of_followers', 'created',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'crowdtangle_score', 'description',
       'description_length', 'hashtag_count', 'last_30_days', 'last_7_days',
       'social_engagement_score', 'time_of_day', 'time_since_last_post',
       'type_link', 'type_photo', 'type_text', 'type_video',
       'within_month_release', 'within_week_release'],
      dtype='object')

In [129]:
#Temporary fixes for youtube
df_list[1].drop(columns='time_of_day', inplace=True)

In [130]:
#Instagram
df_list[3].reset_index(inplace=True)
df_list[3].rename(columns = {'index': 'Unnamed: 0'}, inplace=True)

In [131]:
#Renaming columns that were not aligned
df_list[2].rename(columns = {'fb_handle_artist_name': 'artist_name'}, inplace=True) #Facebook
df_list[0].rename(columns = {'Twitter_Handle': 'artist_name'}, inplace=True) #Twitter

In [132]:
#Adding Dummy columns to indicate which dataset it is
df_list[0]['Facebook'] = 0
df_list[0]['Instagram'] = 0
df_list[0]['Twitter'] = 1
df_list[0]['Youtube'] = 0

df_list[1]['Facebook'] = 0
df_list[1]['Instagram'] = 0
df_list[1]['Twitter'] = 0
df_list[1]['Youtube'] = 1

df_list[2]['Facebook'] = 1
df_list[2]['Instagram'] = 0
df_list[2]['Twitter'] = 0
df_list[2]['Youtube'] = 0

df_list[3]['Facebook'] = 0
df_list[3]['Instagram'] = 1
df_list[3]['Twitter'] = 0
df_list[3]['Youtube'] = 0

In [133]:
fb = df_list[2].merge(artists, left_on='artist_name', right_on='facebook')
instagram = df_list[3].merge(artists, left_on='artist_name', right_on='instagram')
twitter = df_list[0].merge(artists, left_on='artist_name', right_on='twitter')
yt = df_list[1].merge(artists, left_on='artist_name', right_on='youtube')
socials = [fb, instagram, twitter, yt]

In [134]:
for s in socials:
    print(s.shape)

(312204, 40)
(425000, 40)
(324396, 40)
(22788, 40)


###### Investigation Start

In [135]:
fb_check = df_list[2].merge(artists, how='outer', left_on='artist_name', right_on='facebook')


In [136]:
fb_check.shape

(312205, 40)

In [137]:
#Delta between full outer join and inner join
fb_check.shape[0] - fb.shape[0]

1

In [138]:
fb_diff = pd.concat([fb, fb_check]).drop_duplicates(keep=False)

In [139]:
fb_diff.head()

,Unnamed: 0,artist_name_x,created,description,crowdtangle_score,count_of_followers,type_link,type_photo,type_text,type_video,...,genre_rock,genre_hip_hop,genre_world_music,genre_r_and_b,genre_electronic,facebook,instagram,twitter,youtube,musicbrainz
312204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0.0,0,0.0,kanyeWestPage,kanyew.est,kanyewest,Kanye West,164f0d73-1234-4e2c-8743-d77bf2191051


In [140]:
#Differences match
fb_diff.shape

(1, 40)

In [141]:
fb_diff2 = fb_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]

In [142]:
#NOTE: THIS is what was missing originally.  Will be blank if re-run
fb_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


##### Instagram Check

In [143]:
in_check = df_list[3].merge(artists, how='outer', left_on='artist_name', right_on='instagram')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - instagram.shape[0])

in_diff = pd.concat([instagram, in_check]).drop_duplicates(keep=False)

(425002, 40)
2


In [144]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,social_engagement_score,type_photo,...,genre_rock,genre_hip_hop,genre_world_music,genre_r_and_b,genre_electronic,facebook,instagram,twitter,youtube,musicbrainz
425000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.0,1,0.0,theweeknd,deleted IG account ~Jun7 2019,theweeknd,The Weeknd,c8b03190-306c-4120-bb0b-6f2ebfc06ea9
425001,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0.0,0,0.0,kanyeWestPage,kanyew.est,kanyewest,Kanye West,164f0d73-1234-4e2c-8743-d77bf2191051


In [145]:
#Differences match
in_diff.shape

(2, 40)

In [146]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


In [147]:
#Twitter
in_check = df_list[0].merge(artists, how='outer', left_on='artist_name', right_on='twitter')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - twitter.shape[0])

in_diff = pd.concat([twitter, in_check]).drop_duplicates(keep=False)


(324396, 40)
0


In [148]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,type_link,type_photo,...,genre_rock,genre_hip_hop,genre_world_music,genre_r_and_b,genre_electronic,facebook,instagram,twitter,youtube,musicbrainz


In [149]:
#Differences match
in_diff.shape

(0, 40)

In [150]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


##### Youtube Check

In [151]:
#Youtube
in_check = df_list[1].merge(artists, how='outer', left_on='artist_name', right_on='youtube')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - yt.shape[0])

in_diff = pd.concat([yt, in_check]).drop_duplicates(keep=False)

(24245, 40)
1457


In [152]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,created_12am-6am,created_12pm-6pm,created_6am-12pm,created_6pm-12am,crowdtangle_score,description,...,genre_rock,genre_hip_hop,genre_world_music,genre_r_and_b,genre_electronic,facebook,instagram,twitter,youtube,musicbrainz
4737,941.0,BRADPAISLEY,0.0,10/23/2008 10:53,0.0,0.0,1.0,0.0,NaN,Brad Paisley - She's Everything - By Michael V...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4738,942.0,BRADPAISLEY,0.0,10/23/2008 10:53,0.0,0.0,1.0,0.0,NaN,Brad Paisley - She's Everything (cover),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4739,943.0,BRADPAISLEY,0.0,10/23/2008 10:54,0.0,0.0,1.0,0.0,NaN,"""Whiskey Lullaby"" (Brad Paisley Cover) My orig...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4740,944.0,BRADPAISLEY,0.0,10/23/2008 10:59,0.0,0.0,1.0,0.0,NaN,Private video,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4741,945.0,BRADPAISLEY,0.0,10/23/2008 12:02,0.0,1.0,0.0,0.0,NaN,"Brad Paisley ft. Keith Urban - ""Start A Band"" ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
#Differences match
in_diff.shape

(1457, 40)

In [154]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,
BRADPAISLEY,NaN,NaN,NaN,NaN
Carlos Vives,NaN,NaN,NaN,NaN
Chris Young,NaN,NaN,NaN,NaN
Luke Combs,NaN,NaN,NaN,NaN
Miranda Lambert,NaN,NaN,NaN,NaN
Tim McGraw,NaN,NaN,NaN,NaN
WillieNelson,NaN,NaN,NaN,NaN


###### investigation stop

In [155]:
df = pd.concat(socials)
df.drop('Unnamed: 0',axis=1,inplace=True)

C:\Users\oldye\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [156]:
df.sample(10).T

,4352,114606,262835,170022,293887,113082,48716,51833,168743,40442
Facebook,0,0,0,1,0,0,0,0,0,1
Instagram,0,1,1,0,0,0,1,1,0,0
Twitter,0,0,0,0,1,1,0,0,1,0
Youtube,1,0,0,0,0,0,0,0,0,0
age,21,28,32,30,21,27,32,50,50,29
artist_name_x,Alan Walker,farrukoofficial,nattinatasha,chrisbrown,thegreatkhalid,PTXofficial,champagnepapi,chayanne,VictorManuelle,GerardoOrtizNet
artist_name_y,Alan Walker,Farruko,Natti Natasha,Chris Brown,Khalid,Pentatonix,Drake,Chayanne,Victor Manuelle,Gerardo Ortiz
count_of_followers,3.21571e+06,1.5127e+06,227526,3.99184e+07,45501,977903,1.51471e+07,2.9946e+06,639909,1.01083e+07
created,5/25/2018 8:10,2015-05-19 23:07:53,2017-03-06 22:37:03,2015-10-15 21:46:04,2017-01-24 20:10:40,2019-05-15 06:32:54,2015-10-30 01:47:07,2017-04-24 12:52:48,2016-07-28 16:33:59,2015-12-29 20:08:23
created_12am-6am,0,0,0,0,0,0,1,0,0,0


In [157]:
print(df.shape)
print("number of rows lost: ", sum - df.shape[0], ", ", (sum - df.shape[0])/sum)

(1084388, 39)
number of rows lost:  1449 ,  0.001334454434689553


In [158]:
df.head(3).T

,0,1,2
Facebook,1,1,1
Instagram,0,0,0
Twitter,0,0,0
Youtube,0,0,0
age,26,26,26
artist_name_x,21Savage,21Savage,21Savage
artist_name_y,21 Savage,21 Savage,21 Savage
count_of_followers,NaN,NaN,NaN
created,2015-05-18 15:24:23,2015-05-25 12:00:00,2015-05-25 18:26:12
created_12am-6am,0,0,0


In [159]:
# df.to_csv('Combined_Social_Data.csv')

In [160]:
# Filtering outliers in FB data
df = df[~((df['Facebook'] == 1) & (df['social_engagement_score'] > 1))]

In [161]:
# Get the social platform name for each dataframe row
def getSocialPlatform(x):
    if x['Facebook'] == 1:
        return 'Facebook'
    elif x['Instagram'] == 1:
        return 'Instagram'
    elif x['Twitter'] == 1:
        return 'Twitter'
    else:
        return 'Youtube'

In [162]:
# Create social platform column
df['social_platform'] = df.apply(lambda x: getSocialPlatform(x), axis = 1)

In [163]:
# Get the mean & standard deviation of social engagement scores per platform and store the results in 3 dictionaries
dict_platform_mean = df.groupby('social_platform')['social_engagement_score'].mean().to_dict()
dict_platform_std = df.groupby('social_platform')['social_engagement_score'].std().to_dict()

In [164]:
# Normalize the social engagement scores
df['normalized_social_engagement_score'] = df.apply(lambda x: (x['social_engagement_score'] - dict_platform_mean[x['social_platform']]) / dict_platform_std[x['social_platform']],axis=1)

In [166]:
# Reading music Brainz data and removing unwanted column
df_musicbrainz = pd.read_csv("./July8Data/musicbrainz_data.csv")
df_musicbrainz = df_musicbrainz.drop("Unnamed: 0",axis=1)

In [167]:
# Converting the type of release_date to datettime for date comparisons
df_musicbrainz['release_date'] = pd.to_datetime(df_musicbrainz['release_date'])

In [168]:
# Filter the release dates before Jan 1st 2006
df_musicbrainz = df_musicbrainz[df_musicbrainz['release_date'] > datetime.date(2006,1,1)]

C:\Users\oldye\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [169]:
# Convert master data 'created' column to datetime for date comparisons
df['created'] = pd.to_datetime(df['created'])

In [170]:
artist_dict = df_musicbrainz.groupby('artist_name')['release_date'].apply(list).to_dict()

In [171]:
# Function to find days between 2 datetimes
from datetime import datetime
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [172]:
# Function to find the minimum difference of days w.r.t. all release dates of an artist
def find_min_diff(artist_name,created):
    lst = artist_dict[artist_name]
    min_days = min([days_between(x.strftime('%Y-%m-%d'),created.strftime('%Y-%m-%d')) for x in lst])
    return min_days

In [173]:
# Calculating the number of days to track release
t1 = time.time()
df['num_days_to_track_release'] = df.apply(lambda x: find_min_diff(x['artist_name_y'],x['created']),axis=1)
print("time taken",(time.time() - t1))

time taken 3354.2252345085144


In [174]:
df.head()

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube,social_platform,normalized_social_engagement_score,num_days_to_track_release
0,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-18 15:24:23,0,...,0,1,0,0,0,5,21 Savage,Facebook,NaN,7
1,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,0
2,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 18:26:12,0,...,0,0,0,0,0,5,21 Savage,Facebook,NaN,0
3,1,0,0,0,26,21Savage,21 Savage,NaN,2015-08-14 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,43
4,1,0,0,0,26,21Savage,21 Savage,NaN,2015-09-05 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,65


In [175]:
# Saving the dataframe to pickle file
df.to_pickle("Combined_Social_Data_Pickle.pkl")

In [176]:
df.shape

(1084355, 42)

In [177]:
df.head()

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube,social_platform,normalized_social_engagement_score,num_days_to_track_release
0,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-18 15:24:23,0,...,0,1,0,0,0,5,21 Savage,Facebook,NaN,7
1,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,0
2,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 18:26:12,0,...,0,0,0,0,0,5,21 Savage,Facebook,NaN,0
3,1,0,0,0,26,21Savage,21 Savage,NaN,2015-08-14 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,43
4,1,0,0,0,26,21Savage,21 Savage,NaN,2015-09-05 12:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,65
